In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import time
# import custom libraries
import sys
sys.path.append("C:\\DATA\\Tasks\\lib\\hk")
import hk_psql
import dash_bootstrap_components

import plotly.express as px  # (version 4.7.0 or higher)
import plotly.graph_objects as go
from plotly.subplots import make_subplots


ADD_DATA = "C:\\DATA\\data\\raw\\mimic4\\lookup\\"

%load_ext autoreload
%autoreload 2

conn = hk_psql.connect_psql('mimic')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


DATABASE CONNECTED !!!


In [ ]:
sel_icustay = [{'id_careunit': 'Medical Intensive Care Unit (MICU)', 'id_los_min': '24', 'id_los_max': '240'},
 {'id_careunit': 'Trauma SICU (TSICU)', 'id_los_min': '24', 'id_los_max': '240'},
 {'id_careunit': 'Surgical Intensive Care Unit (SICU)', 'id_los_min': '24', 'id_los_max': '240'}]

sel_icustay = [
    {'id_careunit': 'ICU', 'id_los_min': '24', 'id_los_max': '100'},
    # {'id_careunit': 'Cardiac Surgery', 'id_los_min': '0', 'id_los_max': '1000'},
]


filters_core = {
        'age':[20,91],
        'gender':['M','F'],
        'icustays':pd.DataFrame.from_dict(sel_icustay),
        'adm_types':[],
        'adm_locs':[],
        'dis_locs':[],

    }
# print(filters)
q_core = hk_psql.bq_core(filters_core)
# print(q_core)
df_filtered_core = hk_psql.query(q_core, conn)
hadm_ids_core = df_filtered_core['hadm_id'].unique().tolist()
print(df_filtered_core)
print('len after core: ',len(hadm_ids_core))

In [ ]:
# **************** STEP 2: filter hosp codes (drg , icd)
'''	icd_code	icd_version	long_title	count
130	3613	9	(Aorto)coronary bypass of three coronary arteries	1074
258	021209W	10	Bypass Coronary Artery, Three Arteries from Aorta with Autologous Venous Tissue, Open Approach	520
'''

filters_codes = {
    'drgcodes': pd.DataFrame(), 
    'diagnoses_icd': pd.read_csv(ADD_DATA+'saved_diag_sepsis.csv'),
    'procedures_icd': pd.DataFrame(),
}


q = hk_psql.bq_codes(filters_codes, hadm_ids_core)
# q = hk_psql.bq_codes(filters_codes, [])
print('sdfds')
if len(filters_codes['procedures_icd'])+len(filters_codes['diagnoses_icd'])+len(filters_codes['drgcodes'])==0:
    # if no code is specified, just ignore query
    df_filtered_codes = df_filtered_core
else:
    df_filtered_codes = hk_psql.query(q, conn)
# print(df_filtered_codes)
# df_filtered_core.to_csv(ADD_DATA+'temp.csv')
hadm_ids_core_code = df_filtered_codes['hadm_id'].unique().tolist()
print('len after code: ', len(hadm_ids_core_code))



In [ ]:
# **************** STEP 3: Hosp filters -> hadm_ids
lab_table = [
{'id_item': 50889, 'id_label': 'C-Reactive Protein', 'id_fluid': 'Blood', 'id_cat': 'Chemistry', 'id_range': '0.0-5.0', 'id_unit': 'mg/L', 'id_condition': 'X>0', 'id_counts': 20}, 
# {'id_item': 51003, 'id_label': 'Troponin T', 'id_fluid': 'Blood', 'id_cat': 'Chemistry', 'id_range': '0.0-0.01', 'id_unit': 'ng/mL', 'id_condition': 'X>0', 'id_counts': 1}, 
# {'id_item': 50963, 'id_label': 'NTproBNP', 'id_fluid': 'Blood', 'id_cat': 'Chemistry', 'id_range': '0.0-177.0', 'id_unit': 'pg/mL', 'id_condition': 'X>0', 'id_counts': 1}, 
# {'id_item': 50924, 'id_label': 'Ferritin', 'id_fluid': 'Blood', 'id_cat': 'Chemistry', 'id_range': '13.0-150.0', 'id_unit': 'ng/mL', 'id_condition': 'X>0', 'id_counts': 1}, 
# {'id_item': 50909, 'id_label': 'Cortisol', 'id_fluid': 'Blood', 'id_cat': 'Chemistry', 'id_range': '2.0-20.0', 'id_unit': 'ug/dL', 'id_condition': 'X>0', 'id_counts': 1},
]       


filters_hosp = {
    'labevents': pd.DataFrame.from_dict(lab_table),
}
filters_hosp = {
    'labevents': pd.DataFrame(),
}


q_hosp = hk_psql.bq_hosp(hadm_ids_core_code, filters_hosp)

df_filtered_hosp = hk_psql.query(q_hosp, conn)
print(df_filtered_hosp)
df_filtered_core = pd.merge(df_filtered_hosp[['hadm_id']], df_filtered_core[['subject_id','hadm_id','anchor_age','gender','careunit','icu_los','stay_id']], how='inner',on=['hadm_id'])
df_filtered = pd.merge(df_filtered_hosp, df_filtered_core[['subject_id','hadm_id','anchor_age','gender','careunit','icu_los','stay_id']], how='inner',on=['hadm_id'])

print(df_filtered_core)
# print(df_filtered_hosp)        
print(df_filtered)

hadm_ids = df_filtered_core['hadm_id'].unique().tolist()
stay_ids = df_filtered_core['stay_id'].unique().tolist()
sub_ids = df_filtered_core['subject_id'].unique().tolist()

print('len after hosp: ',len(hadm_ids))

In [ ]:
cohort_name = 'sepsis'
hk_psql.save_cohort(hadm_ids, cohort_name)

# load a cohort

In [2]:
# load a cohort
cohort_name = 'sepsis'



schema_core = cohort_name+'_mimic_core'
df_overview = hk_psql.summary(schema_core, conn)
hadm_ids = df_overview['hadm_id'].unique()

q=f"""
    select * from mimic_core.patients t1
    inner join {schema_core}.admissions t2
    on t1.subject_id = t2.subject_id
    where t2.hadm_id in ({str(hadm_ids.tolist())[1:-1]})
    
    """
# print(q)

df_patients = hk_psql.query(q, conn)

q=f"""
        select * from {schema_core}.transfers         
        """
# print(q)

df_transfers = hk_psql.query(q, conn)

q=f"""
        select * from {schema_core}.admissions         
        """
# print(q)

df_admissions = hk_psql.query(q, conn)

q=f"""
        select * from {cohort_name+'_mimic_icu'}.icustays         
        """
# print(q)

df_icustay = hk_psql.query(q, conn)

# LAB EVENTS
q=f"""
select * from {cohort_name+'_mimic_hosp'}.labevents         
"""

df_le = hk_psql.query(q, conn)

# icd_procedure code
q=f"""
select * from {cohort_name+'_mimic_hosp'}.procedures_icd         
"""
df_icd_proc = hk_psql.query(q, conn)

# icd_diagnoses code
q=f"""
select * from {cohort_name+'_mimic_hosp'}.diagnoses_icd         
"""
df_icd_diag = hk_psql.query(q, conn)


# CHART EVENTS
q=f"""
select * from {cohort_name+'_mimic_icu'}.chartevents         
"""

df_che = hk_psql.query(q, conn)

# PROCEDURE EVENTS
q=f"""
select * from {cohort_name+'_mimic_icu'}.procedureevents         
"""

df_pe = hk_psql.query(q, conn)

# Medications
q=f"""
select * from {cohort_name+'_mimic_hosp'}.emar         
"""

df_emar = hk_psql.query(q, conn)

df_patients = df_patients.loc[:,~df_patients.columns.duplicated()]
print(df_patients['subject_id'])
print(df_transfers['subject_id'])


cols_to_use = [x for x in df_transfers.columns.tolist() if x not in df_patients.columns.tolist()]
cols_to_use.append('subject_id')
print(cols_to_use)
df_demo = pd.merge(df_patients, df_transfers[cols_to_use], how='inner',  on='subject_id')

print(df_demo)
df_le_main = df_le.copy()

--- 0.02393960952758789 seconds ---
--- 0.24132084846496582 seconds ---
--- 0.057428598403930664 seconds ---
--- 0.019887924194335938 seconds ---
--- 0.03173708915710449 seconds ---
--- 21.110605716705322 seconds ---
--- 0.06666374206542969 seconds ---
--- 0.16445112228393555 seconds ---
--- 608.9272890090942 seconds ---
--- 1.495635747909546 seconds ---
--- 20.965418338775635 seconds ---
0       10545747
1       12823483
2       14122388
3       14122388
4       14819767
          ...   
2805    19793638
2806    19836795
2807    19921864
2808    19962418
2809    19990427
Name: subject_id, Length: 2810, dtype: int64
0        12567919
1        13758211
2        18109635
3        14371035
4        16151261
           ...   
15664    16800514
15665    19439124
15666    17851873
15667    17513048
15668    10352268
Name: subject_id, Length: 15669, dtype: int64
['transfer_id', 'eventtype', 'careunit', 'intime', 'outtime', 'subject_id']
       subject_id gender  anchor_age  anchor_year anchor

# cohort vis

In [3]:
# what is the latency between charttime and storetime?
list_lab = ['Lactate','Troponin T','C-Reactive Protein','Potassium','Sodium','Creatinine','Urea Nitrogen',]



df_temp = df_le.copy()

df_temp['diff'] = ((df_temp['storetime']-df_temp['charttime']) / np.timedelta64(1, 'm'))
# choose the hid with highest number of measurements


df_temp = df_temp.groupby(['itemid'])['diff'].describe().reset_index()
df_temp = df_temp.merge(pd.read_csv(ADD_DATA+'d_labitems.csv')[['itemid','label']], on='itemid').drop_duplicates('label')


# plot median of measurement intervals
q= df_temp['label'].isin(list_lab)
df_temp[q]
fig=px.bar(df_temp[q], x='label', y='50%')
fig.show()




,itemid,count,mean,std,min,25%,50%,75%,max,label
13,50813,41238.0,6.185533,165.867381,0.0,2.0,3.0,5.0,31484.0,Lactate
97,50889,1023.0,277.562072,337.072382,39.0,96.5,149.0,354.5,4903.0,C-Reactive Protein
131,50912,73634.0,84.658609,43.650391,-13.0,59.0,74.0,98.0,1807.0,Creatinine
249,50971,75875.0,83.777489,42.770169,-13.0,58.0,74.0,97.0,1807.0,Potassium
267,50983,75670.0,84.007216,42.794829,-13.0,59.0,74.0,97.0,1807.0,Sodium
297,51003,6229.0,136.845722,163.132337,27.0,64.0,85.0,138.0,3917.0,Troponin T
299,51006,73480.0,82.898054,41.762948,-13.0,58.0,73.0,96.0,1807.0,Urea Nitrogen


In [4]:
# plot charttime and storetime
list_lab = ['Lactate','Troponin T','C-Reactive Protein']
list_lab = ['C-Reactive Protein']
list_lab = ['Troponin T']


df_temp = df_le.copy()
df_temp['diff'] = ((df_temp['storetime']-df_temp['charttime']) / np.timedelta64(1, 'm'))
df_temp = df_temp.merge(pd.read_csv(ADD_DATA+'d_labitems.csv')[['itemid','label']].drop_duplicates(subset='itemid'), on='itemid')
df_temp = df_temp[df_temp['label'].isin(list_lab)]

# choose the hid with highest number of measurements



# plot median of measurement intervals

df_temp.groupby(['hadm_id','itemid'], as_index=False).size().reset_index().sort_values(by='size',ascending=False)







hid = 29765862
df_temp = df_temp[df_temp['hadm_id']==hid].sort_values(by=['itemid','charttime'])



fig = make_subplots(rows=len(list_lab), cols=1,shared_xaxes=True,subplot_titles=df_temp['label'].unique())

for ii,col in enumerate(list_lab):
    q = df_temp['label']==col
    
    t_chart=df_temp.loc[q,'charttime']
    t_store=df_temp.loc[q,'storetime']
    y=df_temp.loc[q,'value'].astype(float)
    

    _ = fig.add_trace(
                    go.Scatter(x=t_chart, y=y,name=col+' charted',mode='lines+markers'),
                    row=ii+1, col=1
                )
    _ = fig.add_trace(
                    go.Scatter(x=t_store, y=y,name=col+' stored',mode='lines+markers'),
                    row=ii+1, col=1
                )
    
    # adding normal range


fig.show()
fig.write_html(ADD_DATA+'temp.html')


,index,hadm_id,itemid,size
101,101,20581253,51003,29
609,609,23365149,51003,27
1251,1251,26835951,51003,26
1446,1446,28015626,51003,26
1729,1729,29765862,51003,25
...,...,...,...,...
1355,1355,27416443,51003,1
182,182,21053903,51003,1
891,891,24929525,51003,1
1350,1350,27385559,51003,1


# vis pat

In [5]:
# step 1) find the patient based on criteria
# use [VI] to find the transfer id

In [6]:
# step 2) visualize 
from IPython.display import display
tid = 39658297
hid = df_transfers[df_transfers['transfer_id']==tid].iloc[0].hadm_id
hid

df2_tr = df_transfers[df_transfers['hadm_id']==hid]
df2_tr

t_in = df2_tr[df2_tr['transfer_id']== tid].intime.values[0]
t_out = df2_tr[df2_tr['transfer_id']== tid].outtime.values[0]
print('t_in, t_out \n ',t_in, t_out)



print("ICD codes")
# df_icd_proc[df_icd_proc['hadm_id'] == hid]
#df_icd_diag[df_icd_diag['hadm_id'] == hid]

q =  (df_emar['hadm_id'] == hid) & (df_emar['charttime'] > t_in) & (df_emar['charttime'] <= t_out)

df_emar[ q ].medication.value_counts()






28080619

,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime
8374,16857350,28080619,35938307,transfer,None,2161-11-30 12:29:22,2161-11-30 12:29:40
8376,16857350,28080619,36422091,discharge,None,2161-12-05 11:45:27,NaT
9320,16857350,28080619,36030997,admit,Trauma SICU (TSICU),2161-11-19 08:07:07,2161-11-30 12:29:22
9340,16857350,28080619,35375614,transfer,Trauma SICU (TSICU),2161-11-30 12:29:40,2161-11-30 21:46:36
9341,16857350,28080619,30418278,transfer,Med/Surg,2161-11-30 21:46:36,2161-12-03 13:14:32
9344,16857350,28080619,39658297,transfer,Medical Intensive Care Unit (MICU),2161-12-03 13:14:32,2161-12-05 11:45:27
9847,16857350,28080619,30238690,ED,Emergency Department,2161-11-19 05:03:00,2161-11-19 08:07:07


t_in, t_out 
  2161-12-03T13:14:32.000000000 2161-12-05T11:45:27.000000000
ICD codes


Sodium Chloride 0.9%  Flush         17
Lactulose                           10
Rocuronium                           8
Insulin                              7
Sodium Bicarbonate                   7
Ipratropium-Albuterol Neb            7
Fentanyl Citrate                     6
Propofol                             5
Penicillin G Potassium               4
NORepinephrine                       4
Piperacillin-Tazobactam              3
Vancomycin                           3
Heparin                              3
Artificial Tears GEL 1%              3
Rifaximin                            3
Clindamycin                          2
Vasopressin                          2
Heparin Flush (10 units/ml)          2
Pantoprazole                         2
EPINEPHrine                          2
Citrate Dextrose 3% (ACD-A) CRRT     1
FoLIC Acid                           1
Multivitamins W/minerals             1
Thiamine                             1
Sodium CITRATE 4%                    1
Acetaminophen            

In [12]:
######################################## select variables
 
dict_vitals = {220045:['HR', [60,90], ['L', 'N', 'H'] ],
               220210:['RR', [10,18], ['L', 'N', 'H'] ],
               220277:['SpO2', [90,101], ['L', 'N'] ],}


dict_labs = {50971:['Potassium',[3.3,5],['L', 'N', 'H']], 
50983:['Sodium',[133,145],['L', 'N', 'H']], 
50902:['Chloride',[96,108],['L', 'N', 'H']], 
51006:['Urea Nitrogen',[6,20],['L', 'N', 'H']], 
50912:['Creatinine',[0.5,1.2],['L', 'N', 'H']], 
50889:['C-Reactive Protein',[0, 5],['L', 'N', 'H']], 


51003:[	'Troponin T',[0,1e-10,0.1,1000],['L', 'N', 'H']],
#50910:[	'Creatine Kinase (CK)'],
50813:[	'Lactate',[0.5,2],['L', 'N', 'H']],}

list_emar = ['NORepinephrine','EPINEPHrine','DOBUTamine','DOPamine','PHENYLEPHrine']
dict_proc = {225792:'Invasive Ventilation', 225802:'Dialysis - CRRT', 220277:'SpO2'}


# Extracting vitals from ICU chartevents
dff_che = df_che[df_che['hadm_id']==hid].query(f"itemid.isin({list(dict_vitals.keys())})").merge(pd.read_csv(ADD_DATA+'d_items.csv')[['itemid','label']], on='itemid')
dff_che = dff_che[['charttime','itemid','valuenum','label']].sort_values('charttime',ascending=True) 
dff_che = dff_che.rename(columns={'valuenum':'value'})
dff_che['type'] = 'vital'
## creating mask for each vital
dff_che['mask'] = np.nan
for itemid, values in dict_vitals.items():
    q = dff_che['itemid']==itemid    
    dff_che.loc[q, 'mask'] = pd.cut(dff_che.loc[q,'value'], bins=[-1e10,*values[1],1e10], labels=['L','N','H'])
    dff_che.loc[q, 'lb'] = values[1][0]
    dff_che.loc[q, 'ub'] = values[1][1]
    
dff_che['time_grid'] = ( (dff_che['charttime'] - t_in) / np.timedelta64(1, 'm')).astype(int)
dff_che = dff_che[dff_che['time_grid']>0]
dff_che = dff_che.drop_duplicates(subset=['itemid','time_grid'],keep='first')
dff_che = dff_che.sort_values(['itemid','charttime'])
dff_che


# Extracting labs from HOSP labevents
dff_le = df_le[df_le['hadm_id']==hid].query(f"itemid.isin({list(dict_labs.keys())})").merge(pd.read_csv(ADD_DATA+'d_labitems.csv')[['itemid','label']], on='itemid')
dff_le = dff_le[['charttime','itemid','value','label']].sort_values('charttime',ascending=True)
dff_le['type'] = 'lab'
## creating mask for each vital
dff_le['mask'] = np.nan
for itemid, values in dict_labs.items():
    q = dff_le['itemid']==itemid
    dff_le.loc[q, 'mask'] = pd.cut(dff_le.loc[q,'value'].astype(float), bins=[-1e10,*values[1],1e10], labels=['L','N','H'])
    dff_le.loc[q, 'lb'] = values[1][0]
    dff_le.loc[q, 'ub'] = values[1][1]

dff_le['time_grid'] = ( (dff_le['charttime'] - t_in) / np.timedelta64(1, 'm')).astype(int)
dff_le = dff_le[dff_le['time_grid']>0]
dff_le = dff_le.drop_duplicates(subset=['itemid','time_grid'],keep='first')
dff_le = dff_le.sort_values(['itemid','charttime'])
dff_le

dff_le_samp = dff_le.copy()
dff_le_samp['type'] = 'lab_samp'
dff_le_samp['mask'] = np.nan
dff_le_samp['label'] = dff_le_samp['label']+'_sam'
dff_le_samp['label2'] = dff_le_samp['label']

dff_emar = df_emar[df_emar['hadm_id']==hid].query(f"medication.isin({list_emar})")
dff_emar = dff_emar[['charttime','medication']].sort_values('charttime',ascending=True)
dff_emar['value'] = 0
dff_emar['value2'] = 1
dff_emar['lb'] = 0
dff_emar['ub'] = 0
dff_emar['label2'] = dff_emar['medication']
dff_emar['type'] = 'med'
dff_emar['time_grid'] = ( (dff_emar['charttime'] - t_in) / np.timedelta64(1, 'm')).astype(int)
dff_emar = dff_emar[dff_emar['time_grid']>0]
dff_emar = dff_emar.rename(columns={'medication':'label'})
dff_emar

df_con = pd.concat([dff_che,dff_le,dff_emar,dff_le_samp],ignore_index=True) # ignore_index is necessary
df_con

,charttime,itemid,value,label,type,mask,lb,ub,time_grid
303,2161-12-03 13:25:00,220045,100.0,Heart Rate,vital,H,60.0,90.0,10
304,2161-12-03 14:00:00,220045,107.0,Heart Rate,vital,H,60.0,90.0,45
305,2161-12-03 15:10:00,220045,97.0,Heart Rate,vital,H,60.0,90.0,115
306,2161-12-03 16:00:00,220045,103.0,Heart Rate,vital,H,60.0,90.0,165
307,2161-12-03 17:00:00,220045,88.0,Heart Rate,vital,N,60.0,90.0,225
...,...,...,...,...,...,...,...,...,...
1046,2161-12-05 05:00:00,220277,93.0,O2 saturation pulseoxymetry,vital,N,90.0,101.0,2385
1047,2161-12-05 06:00:00,220277,84.0,O2 saturation pulseoxymetry,vital,L,90.0,101.0,2445
1048,2161-12-05 07:00:00,220277,100.0,O2 saturation pulseoxymetry,vital,N,90.0,101.0,2505
1049,2161-12-05 08:00:00,220277,100.0,O2 saturation pulseoxymetry,vital,N,90.0,101.0,2565


ValueError: Bin labels must be one fewer than the number of bin edges

In [15]:
selected_types = ['vital','lab','med']
selected_types = ['med','vital','lab','lab_samp']
selected_types=['lab']
# plot timeseries of vitals and labs

df_temp = df_con[df_con['type'].isin(selected_types)].sort_values(['type','label','time_grid'])
df_temp['value'] = df_temp['value'].astype(float)
df_temp['value2'] = 0

df_temp[df_temp['label2']=='Lactate_sam']
df_temp[df_temp['label2']=='Lactate N']

for i,col in enumerate(df_temp['label'].unique()):
    if col.endswith('sam'):
        continue
    print(i, col)
    q= df_temp['label']==col
    
    val_mean = df_temp.loc[q,'value'].mean()
    val_std = df_temp.loc[q,'value'].std()
    df_temp.loc[q,'value_n'] = (df_temp.loc[q,'value'] - val_mean) / val_std + i*4
    # print(df_temp.loc[q,'value'].mean())
    # print(df_temp.loc[q,'value'].std())


    mask = df_temp.loc[q,'mask']

    

    i_N2H = (mask=='H') & (mask.shift(1,fill_value='N')=='N')
    i_H2N = (mask=='N') & (mask.shift(1,fill_value='N')=='H')
    i_N2L = (mask=='L') & (mask.shift(1,fill_value='N')=='N')
    i_L2N = (mask=='N') & (mask.shift(1,fill_value='N')=='L')

    

    df_temp.loc[mask[i_N2H].index, 'label2'] = col+' H'
    df_temp.loc[mask[i_N2H].index, 'value2'] = 1
    df_temp.loc[mask[i_H2N].index, 'label2'] = col+' N'
    df_temp.loc[mask[i_H2N].index, 'value2'] = 1
    df_temp.loc[mask[i_N2L].index, 'label2'] = col+' L'
    df_temp.loc[mask[i_N2L].index, 'value2'] = 1
    df_temp.loc[mask[i_L2N].index, 'label2'] = col+' N'
    df_temp.loc[mask[i_L2N].index, 'value2'] = 1




df_temp.loc[df_temp['type']=='med', 'value2'] = 1



# plt.figure()
# df_temp.plot();

# fig = make_subplots(rows=2, cols=1, shared_xaxes=True)    
    
# t = df_temp['time_grid']
# y = df_temp['value_n']
# _ = fig.add_trace(go.Scatter(x=t, y=y, mode='markers+lines'), row=1, col=1)

df_temp
print('****************************************plot timeseries in one plot') 
fig = px.line(df_temp, x='time_grid', y ='value_n',color='label', markers=True)
_ = fig.update_xaxes(range=[0, df_temp['time_grid'].max()+1])
# _ = fig.update_layout(showlegend=False)
fig.show()
fig.write_html(ADD_DATA+'temp.html')


print('****************************************plot timeseries in subplots') 
n = len(df_temp['label'].unique())
print(df_temp['label'].unique())
print(df_con['type'].unique())
fig = make_subplots(rows=n, cols=1,shared_xaxes=True,subplot_titles=df_temp['label'].unique())

for ii,col in enumerate(df_temp['label'].unique()):

    q = df_temp['label']==col
    type = df_temp.loc[q,'type'].iloc[0]
    mode_type = ['markers' if type=='med' else 'lines+markers']
    t=df_temp.loc[q,'time_grid']
    y=df_temp.loc[q,'value']
    #t = round(t/60)
    #print(y,round(y/60))

    _ = fig.add_trace(
                    go.Scatter(x=t, y=y,name=col, mode= mode_type[0],),
                    row=ii+1, col=1
                )
    
    # adding normal range
    lb = df_temp.loc[q,'lb'].iloc[0]
    ub = df_temp.loc[q,'ub'].iloc[0]
    _ = fig.add_hrect(y0=lb, y1=ub, line_width=0, fillcolor="green", opacity=0.2,row=ii+1, col=1)


#    _ = fig.update_yaxes(title_text=col, row=ii+1, col=1)
_ = fig.update_traces(showlegend=False)
fig.show()
#fig.write_html(ADD_DATA+'temp.html')



print('****************************************plot ES data') 
fig = px.scatter(df_temp.dropna(subset='label2').sort_values(['type','label2']),x='time_grid',y='label2',color='label2')
_ = fig.update_traces(marker=dict(size=12,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                                        #symbol='diamond',
                  selector=dict(mode='markers'))
_ = fig.update_xaxes(range=[-60, df_temp['time_grid'].max()+60])
_ = fig.update_layout(showlegend=False)

fig.show()
# fig.write_html(ADD_DATA+'temp.html')


,charttime,itemid,value,label,type,mask,lb,ub,time_grid,value2,label2


,charttime,itemid,value,label,type,mask,lb,ub,time_grid,value2,label2


0 Chloride
1 Creatinine
2 Lactate
3 Potassium
4 Sodium
5 Urea Nitrogen


,charttime,itemid,value,label,type,mask,lb,ub,time_grid,value2,label2,value_n
156,2161-12-03 17:36:00,50902.0,115.0,Chloride,lab,H,96.0,108.0,261,1,Chloride H,0.920358
157,2161-12-03 21:05:00,50902.0,115.0,Chloride,lab,H,96.0,108.0,470,0,NaN,0.920358
158,2161-12-04 03:41:00,50902.0,113.0,Chloride,lab,H,96.0,108.0,866,0,NaN,-0.613572
159,2161-12-04 13:17:00,50902.0,114.0,Chloride,lab,H,96.0,108.0,1442,0,NaN,0.153393
160,2161-12-05 02:13:00,50902.0,112.0,Chloride,lab,H,96.0,108.0,2218,0,NaN,-1.380537
161,2161-12-03 17:36:00,50912.0,1.7,Creatinine,lab,H,0.5,1.2,261,1,Creatinine H,3.630988
162,2161-12-03 21:05:00,50912.0,1.8,Creatinine,lab,H,0.5,1.2,470,0,NaN,4.092253
163,2161-12-04 03:41:00,50912.0,1.8,Creatinine,lab,H,0.5,1.2,866,0,NaN,4.092253
164,2161-12-04 13:17:00,50912.0,1.5,Creatinine,lab,H,0.5,1.2,1442,0,NaN,2.708456
165,2161-12-05 02:13:00,50912.0,2.1,Creatinine,lab,H,0.5,1.2,2218,0,NaN,5.476050


****************************************plot timeseries in one plot


****************************************plot timeseries in subplots
['Chloride' 'Creatinine' 'Lactate' 'Potassium' 'Sodium' 'Urea Nitrogen']
['vital' 'lab' 'med' 'lab_samp']


****************************************plot ES data


In [ ]:
df_temp[df_temp['label2']=='Lactate_sam']
df_temp[df_temp['label2']=='Lactate N']

In [ ]:
# Extracting labs from HOSP labevents
dff_le = df_le[df_le['hadm_id']==hid].query(f"itemid.isin({list(dict_labs.keys())})").merge(pd.read_csv(ADD_DATA+'d_labitems.csv')[['itemid','label']], on='itemid')
dff_le = dff_le[['charttime','itemid','value','label']].sort_values('charttime',ascending=True)

## creating mask for each vital
dff_le['mask'] = np.nan
for itemid, values in dict_labs.items():
    print(itemid,values[0],values[1])
    q = dff_le['itemid']==itemid
    dff_le.loc[q, 'mask'] = pd.cut(dff_le.loc[q,'value'].astype(float), bins=values[1], labels=values[2])
dff_le.loc[q, 'mask']

dff_emar = df_emar[df_emar['hadm_id']==hid].query(f"medication.isin({list_emar})")
dff_emar = dff_emar[['charttime','medication']].sort_values('charttime',ascending=True)
dff_emar['value'] = 1
dff_emar.rename(columns={'medication':'label'})
dff_emar


In [ ]:
#fig = make_subplots(rows=10, cols=1, shared_xaxes=True)
#dff_che = df_che[df_che['hadm_id']==hid].sort_values('charttime',ascending=True)
#dff_le = df_le[df_le['hadm_id']==hid].sort_values('charttime',ascending=True)

########## plot VITALS
marker_style = dict(
            color='red',
            symbol = 'triangle-up',
            size=10,
            line=dict(
                color='MediumPurple',
                width=1
            ))
fig = make_subplots(rows=len(dict_vitals), cols=3, shared_xaxes=True)
ii=1
for k,v in dict_vitals.items():
    #print(k,v)
    q =  (dff_che['charttime'] > t_in) & (dff_che['charttime'] <= t_out)
    t = dff_che.loc[q].query(f"itemid=={k}").charttime
    y = dff_che.loc[q].query(f"itemid=={k}").valuenum
    mask = dff_che.loc[q].query(f"itemid=={k}")['mask']
    i_N2H = (mask=='H') & (mask.shift(1)=='N')
    i_H2N = (mask=='N') & (mask.shift(1)=='H')
    i_N2L = (mask=='L') & (mask.shift(1)=='N')
    i_L2N = (mask=='N') & (mask.shift(1)=='L')
    
    
    _ = fig.add_traces(go.Scatter(x=t, y=y, name=v[0], mode='markers+lines'), rows=ii, cols=1)
    _ = fig.add_traces(go.Scatter(x=t, y=mask, name=v[0], mode='markers'), rows=ii, cols=2)
    _ = fig.update_yaxes(categoryorder='array', categoryarray= ['L','N','H'])

    _ = fig.add_traces([go.Scatter(x=t[i_N2H], y=y[i_N2H]*0, mode='markers', marker_symbol = 'triangle-up', marker_color = 'red', marker_size = 7),
                        go.Scatter(x=t[i_H2N], y=y[i_H2N]*0, mode='markers', marker_symbol = 'triangle-down', marker_color = 'green', marker_size = 7),
                        go.Scatter(x=t[i_N2L], y=y[i_N2L]*0, mode='markers', marker_symbol = 'triangle-down', marker_color = 'blue', marker_size = 7),
                        go.Scatter(x=t[i_L2N], y=y[i_L2N]*0, mode='markers', marker_symbol = 'triangle-up', marker_color = 'green', marker_size = 7),
                        
                        ], rows=ii, cols=3)
    

    #_ = fig.add_traces(go.Scatter(x=t, y=y*0, name=v[0], mode='markers'), rows=ii, cols=3)
    ii=ii+1
fig.show()

fig.write_html(ADD_DATA+'temp.html')



In [ ]:
########## plot LABS
fig = make_subplots(rows=len(dict_labs), cols=3, shared_xaxes=True)
ii=1
for k,v in dict_labs.items():
    #print(k,v)
    q =  (dff_le['charttime'] > t_in) & (dff_le['charttime'] <= t_out)
    
    t = dff_le.loc[q].query(f"itemid=={k}")['charttime']
    y = dff_le.loc[q].query(f"itemid=={k}")['value'].astype(float)
    mask = dff_le.loc[q].query(f"itemid=={k}")['mask']

    
    i_N2H = (mask=='H') & (mask.shift(1)=='N')
    i_H2N = (mask=='N') & (mask.shift(1)=='H')
    i_N2L = (mask=='L') & (mask.shift(1)=='N')
    i_L2N = (mask=='N') & (mask.shift(1)=='L')
    # print(t[i_N2H],y[i_N2H]*0)
    # print(t[i_H2N], y[i_H2N]*0)
    #term
    _ = fig.add_traces(go.Scatter(x=t, y=y, name=v[0], mode='markers+lines'), rows=ii, cols=1)

   # _ = fig.add_traces(go.Scatter(x=t, y=mask, name=v[0], mode='markers'), rows=ii, cols=2)
    _ = fig.add_traces(go.Scatter(x=t, y=y*0, name=v[0], mode='markers'), rows=ii, cols=2)
#    _ = fig.update_yaxes(categoryorder='array', categoryarray= ['L','N','H'])

    _ = fig.add_traces([go.Scatter(x=t[i_N2H], y=y[i_N2H]*0, mode='markers', marker_symbol = 'triangle-up', marker_color = 'red', marker_size = 7),
                        go.Scatter(x=t[i_H2N], y=y[i_H2N]*0, mode='markers', marker_symbol = 'triangle-down', marker_color = 'green', marker_size = 7),
                        go.Scatter(x=t[i_N2L], y=y[i_N2L]*0, mode='markers', marker_symbol = 'triangle-down', marker_color = 'blue', marker_size = 7),
                        go.Scatter(x=t[i_L2N], y=y[i_L2N]*0, mode='markers', marker_symbol = 'triangle-up', marker_color = 'green', marker_size = 7),
                        
                        ], rows=ii, cols=3)

    #_ = fig.add_traces(go.Scatter(x=t, y=y*0, name=v[0], mode='markers'), rows=ii, cols=3)
    ii=ii+1
print('hi')
#_ = fig.update_layout(height=1000, title_text="Side By Side Subplots")

fig.show()
fig.write_html(ADD_DATA+'temp.html')


In [ ]:
########## plot LABS
fig = make_subplots(rows=len(dict_labs), cols=3, shared_xaxes=True)
ii=1
for k,v in dict_labs.items():
    #print(k,v)
    q =  (dff_le['charttime'] > t_in) & (dff_le['charttime'] <= t_out)
    
    t = dff_le.loc[q].query(f"itemid=={k}")['charttime']
    y = dff_le.loc[q].query(f"itemid=={k}")['value']
    mask = dff_le.loc[q].query(f"itemid=={k}")['mask']

    # i_N2H = (mask=='H') & (mask.shift(1)=='N')
    # i_H2N = (mask=='N') & (mask.shift(1)=='H')
    # i_N2L = (mask=='L') & (mask.shift(1)=='N')
    # i_L2N = (mask=='N') & (mask.shift(1)=='L')
    
    _ = fig.add_traces(go.Scatter(x=t, y=y, name=v[0], mode='markers+lines'), rows=ii, cols=1)
    #_ = fig.add_traces(go.Scatter(x=t, y=mask, name=v[0], mode='markers'), rows=ii, cols=2)
    #_ = fig.update_yaxes(categoryorder='array', categoryarray= ['L','N','H'])

    # _ = fig.add_traces([
    #                     go.Scatter(x=t[i_N2H], y=i_N2H, mode='markers'),
    #                     go.Scatter(x=t[i_H2N], y=i_H2N, mode='markers'),
    #                     go.Scatter(x=t[i_N2L], y=i_N2L, mode='markers'),
    #                     go.Scatter(x=t[i_L2N], y=i_L2N, mode='markers'),
                        
    #                     ], rows=ii, cols=3)

    #_ = fig.add_traces(go.Scatter(x=t, y=y*0, name=v[0], mode='markers'), rows=ii, cols=3)
    ii=ii+1
print('hi')
#_ = fig.update_layout(height=1000, title_text="Side By Side Subplots")

fig.show()

In [ ]:
fig = make_subplots(rows=10, cols=1, shared_xaxes=True)
dff_che = df_che[df_che['hadm_id']==hid].sort_values('charttime',ascending=True)
dff_le = df_le[df_le['hadm_id']==hid].sort_values('charttime',ascending=True)
dff_pe = df_pe[df_le['hadm_id']==hid].sort_values('starttime',ascending=True)


fig = make_subplots(rows=len(dict_vitals), cols=3, shared_xaxes=True)
ii=1
for k,v in dict_vitals.items():
    #print(k,v)
    t = dff_che.query(f"itemid=={k}").charttime
    y = dff_che.query(f"itemid=={k}").valuenum
    #y = (pd.cut(y,v[1], labels=v[2]))
    
    _ = fig.add_traces(go.Scatter(x=t, y=y, name=v[0]), rows=ii, cols=1)
    
    ii=ii+1

for k,v in dict_labs.items():
    #print(k,v)
    lb =  dff_le.query(f"itemid=={k}").ref_range_lower.iloc[0]
    ub =  dff_le.query(f"itemid=={k}").ref_range_upper.iloc[0]

    t = dff_le.query(f"itemid=={k}").charttime
    y = dff_le.query(f"itemid=={k}").valuenum
    #print( dff_le.query(f"itemid=={k}"))
    #y = (pd.cut(y,[-10000,lb,ub,10000], labels=['L','N','H']))

    
    _ = fig.add_traces(go.Scatter(x=t, y=y, mode='markers', name=v), rows=ii, cols=1)
    
    ii=ii+1

y=0
for k,v in dict_proc.items():
    #print(k,v)
    t1 = dff_pe.query(f"itemid=={k}").starttime
    t2 = dff_pe.query(f"itemid=={k}").endtime
    _ = fig.add_traces(go.Scatter(x=[t1, t2], y=[y, y], mode='markers+lines', name=v), rows=ii, cols=1)
    y=y+1

y=0
for med_name in list_emar:
    #print(med_name)
    med = df_emar.query(f"hadm_id=={i} and medication=='{med_name}'").sort_values('charttime', ascending=True)
    t = med.charttime
    #print(t)
    _ = fig.add_traces(go.Scatter(x=t, y=np.ones(len(t)) * y, mode='markers', name=med_name), rows=ii+1, cols=1)
    y=y+1

#fig.update_layout(height=600, width=800, title_text="Side By Side Subplots")
fig.show()

fig.write_html(ADD_DATA+'data_desc.html')


## data description

In [ ]:
dict_vitals = {220045:['HR', [0,60,90,200], ['L', 'N', 'H'] ],
               220210:['RR', [0,10,18,60], ['L', 'N', 'H'] ],
               220277:['SpO2', [0,90,101], ['L', 'N'] ],}
dict_labs = {50971:'Potassium', 
51003:	'Troponin T',
50910:	'Creatine Kinase (CK)',
#50911:	'Creatine Kinase, MB Isoenzyme',
50813:	'Lactate',}

list_emar = ['Insulin','Fentanyl Citrate','Dexmedetomidine','EPINEPHrine', 'Heparin']

dict_proc = {225792:'Invasive Ventilation', 225802:'Dialysis - CRRT', 220277:'SpO2'}


df_temp= pd.read_csv(ADD_DATA+'d_items.csv')

df_temp = pd.merge(df_pe,df_temp, on='itemid')

temp = df_temp.label.value_counts()


i = df_demo['hadm_id'].iloc[20]



df_temp[df_temp['hadm_id']==i]

med = df_emar.query(f"hadm_id=={i} ")


In [ ]:
fig = make_subplots(rows=10, cols=1, shared_xaxes=True)
dff_che = df_che[df_che['hadm_id']==i].sort_values('charttime',ascending=True)
dff_le = df_le[df_le['hadm_id']==i].sort_values('charttime',ascending=True)
dff_pe = df_pe[df_le['hadm_id']==i].sort_values('starttime',ascending=True)

ii=1
for k,v in dict_vitals.items():
    #print(k,v)
    t = dff_che.query(f"itemid=={k}").charttime
    y = dff_che.query(f"itemid=={k}").valuenum
    #y = (pd.cut(y,v[1], labels=v[2]))
    
    _ = fig.add_traces(go.Scatter(x=t, y=y, name=v[0]), rows=ii, cols=1)
    
    ii=ii+1

for k,v in dict_labs.items():
    #print(k,v)
    lb =  dff_le.query(f"itemid=={k}").ref_range_lower.iloc[0]
    ub =  dff_le.query(f"itemid=={k}").ref_range_upper.iloc[0]

    t = dff_le.query(f"itemid=={k}").charttime
    y = dff_le.query(f"itemid=={k}").valuenum
    #print( dff_le.query(f"itemid=={k}"))
    #y = (pd.cut(y,[-10000,lb,ub,10000], labels=['L','N','H']))

    
    _ = fig.add_traces(go.Scatter(x=t, y=y, mode='markers', name=v), rows=ii, cols=1)
    
    ii=ii+1

y=0
for k,v in dict_proc.items():
    #print(k,v)
    t1 = dff_pe.query(f"itemid=={k}").starttime
    t2 = dff_pe.query(f"itemid=={k}").endtime
    _ = fig.add_traces(go.Scatter(x=[t1, t2], y=[y, y], mode='markers+lines', name=v), rows=ii, cols=1)
    y=y+1

y=0
for med_name in list_emar:
    #print(med_name)
    med = df_emar.query(f"hadm_id=={i} and medication=='{med_name}'").sort_values('charttime', ascending=True)
    t = med.charttime
    #print(t)
    _ = fig.add_traces(go.Scatter(x=t, y=np.ones(len(t)) * y, mode='markers', name=med_name), rows=ii+1, cols=1)
    y=y+1

#fig.update_layout(height=600, width=800, title_text="Side By Side Subplots")
fig.show()

fig.write_html(ADD_DATA+'data_desc.html')


## basic vis

In [ ]:

df_temp = df_transfers.groupby(['hadm_id','careunit']).size().reset_index(name='count').sort_values(by=['hadm_id','careunit','count',],ascending=False)

df_temp = df_temp.groupby(['careunit','count']).size().reset_index(name='size').sort_values(by='size',ascending=False).iloc[:]
df_temp
df_temp['size'] = df_temp['size']/len(hadm_ids) * 100


# FIGURE: frequency of multiple careunits
px.histogram(df_temp, y = 'size', x='careunit', color='count',
#labels={'size':'%'},
 barmode='group',
 title = "frequency of multiple careunits")



In [ ]:
# FIGURE: top-10 frequent care units

px.histogram(df_temp, y = 'size', x='careunit',
#labels={'size':'%'},
 barmode='group',
 title = "frequency of careunits")

# All timeline visualization

In [ ]:
from datetime import datetime, timedelta

date_time_str = '01/01/00 01:55:19'

date_origin = datetime.strptime(date_time_str, '%d/%m/%y %H:%M:%S')

df2_transfers = df_transfers
df2_transfers['detail'] = df2_transfers['careunit']
df2_transfers = df2_transfers.rename(columns={'careunit':'label'})
# drop outtime=nan
df2_transfers = df2_transfers.dropna(subset='outtime')
df2_transfers.dtypes


In [ ]:
# icd-proc codes CABG 

from datetime import datetime, timedelta
df_d_icd_proc = pd.read_csv(ADD_DATA+'d_icd_procedures.csv')



df_icd_CABG = pd.read_csv(ADD_DATA+'saved_proc_cabg.csv')
df_icd_CABG = pd.DataFrame()

temp = df_icd_CABG['icd_code'].tolist()
df_icd_CABG = df_icd_proc.query(f"icd_code.isin({temp})")
df_icd_CABG = pd.merge(df_icd_CABG, df_d_icd_proc,on=['icd_code', 'icd_version'])

df_icd_CABG['intime'] = df_icd_CABG ['chartdate'] 
df_icd_CABG['outtime'] =  df_icd_CABG ['chartdate'] + timedelta(days=1)

df_icd_CABG['detail'] = df_icd_CABG['icd_code'] + '/' +df_icd_CABG['long_title']



df_icd_CABG['label']='CABG'

df_icd_CABG = df_icd_CABG[['hadm_id','label','intime','outtime','detail','icd_code','icd_version']]

print(df_icd_CABG.dtypes)

In [ ]:
# icd-diag codes containing "myocardial infarction"  except "old"

df_d_icd_diag = pd.read_csv(ADD_DATA+'d_icd_diagnoses.csv')

df2_icd_diag = pd.merge(df_icd_diag, df_d_icd_diag,on=['icd_code', 'icd_version'])

df_icd_sepsis = df2_icd_diag.query(f"long_title.str.contains('septic shock',case=False)")

df_icd_sepsis['label'] = 'sepsis'
df_icd_sepsis['detail'] = df_icd_sepsis['icd_code'] + '/' +df_icd_sepsis['long_title']
df_icd_sepsis['intime'] = date_origin - timedelta(days=3)
df_icd_sepsis['outtime'] = date_origin - timedelta(days=2)
df_icd_sepsis = df_icd_sepsis[['hadm_id','label','detail','intime','outtime']]
print(df_icd_sepsis.dtypes)

# FIGURE: plot most frequent sepsis-related codes in the cohort
temp = df_icd_sepsis.groupby('detail').size().reset_index(name='size').sort_values(by='size',ascending=False)
fig = px.bar(temp,x='detail', y='size', title = 'most frequent sepsis-related codes')
fig.show()
fig.write_html(ADD_DATA+'sepsis-related codes.html')
print(len(df_icd_sepsis))


In [ ]:
# "cath lab" as outcome from icu.procedures

# 225430	Cardiac Cath
# 229577	Cath Lab (Received)
# 229578	Cath Lab (Sent)
list_itemids = [
225430,
229577,
229578,
]

dict_cathlab = {
225430	: 'Cardiac Cath',
229577	: 'Cath Lab (Received)',
229578	: 'Cath Lab (Sent)',
}

df2_pe = df_pe.query(f"itemid.isin({list_itemids})")

df2_pe = df2_pe[['hadm_id','starttime','endtime','itemid']]
df2_pe = df2_pe.rename(columns={'starttime':'intime','endtime':'outtime','itemid':'label'})
df2_pe['outtime'] = df2_pe['outtime']+timedelta(days=1)

df2_pe = df2_pe.replace({'label': dict_cathlab})

df2_pe['detail'] = df2_pe['label']

print(df2_pe.dtypes)
print(df2_pe)


In [ ]:
df_concat = pd.concat([df2_transfers, df_icd_sepsis, df2_pe])

df_concat.dtypes

In [ ]:
df_concat['label'].value_counts()
df_concat = pd.merge(df_concat,df_admissions[['hadm_id','admittime']], on='hadm_id')


In [ ]:


# select a subset
#df_concat=df_concat.query(f"careunit=='Cardiac Surgery' or careunit.str.endswith('(CVICU)') or careunit=='CABG' or careunit.isin({list_itemids}) ")

# select a subset that has Cath lab
sub_hadm_ids = df_concat[df_concat['label'].str.contains('Cath').fillna(False)]['hadm_id'].unique().tolist()
df2_concat = df_concat[df_concat['hadm_id'].isin(sub_hadm_ids)]

df2_concat ['intime'] = (df2_concat ['intime'] - df2_concat ['admittime']) + date_origin
df2_concat ['outtime'] = (df2_concat ['outtime'] - df2_concat ['admittime']) +date_origin
q_sepsis = df2_concat['label']=='sepsis'
df2_concat.loc [q_sepsis, 'intime'] = date_origin - timedelta(days=3)
df2_concat.loc [q_sepsis, 'outtime'] = date_origin - timedelta(days=2)
df2_concat = df2_concat.sort_values(['hadm_id','intime']).iloc[:] 




In [ ]:
print(f"total Sepsis patients: {len(df_transfers['hadm_id'].unique())}")
print(f"total number with an outcome: {len(df2_concat['hadm_id'].unique())}")

df2_concat['label'].value_counts()

In [ ]:

# assign hadm_id to a unique list [1: n_adm]
L1 = df2_concat['hadm_id'].unique().tolist()
L2 = list(range(1,len(hadm_ids)+1))
mapper=dict(zip(L1, L2))
df2_concat['id'] = df2_concat['hadm_id']
df2_concat = df2_concat.replace({"id": mapper})

df2_concat['id'] =df2_concat['id']
list_temp = [*list(dict_cathlab.values()),'']
q = df2_concat['label'].isin(list_temp)
df2_concat.loc[q,'id'] = df2_concat.loc[q,'id'] + 0.2


df2_concat = df2_concat[df2_concat.label != 'Emergency Department']
df2_concat = df2_concat.dropna(subset='label')

In [ ]:
# define colors
df2_concat['color'] = 'Other units'

q = df2_concat['label'].str.contains('ICU').fillna(False)
df2_concat.loc[q,'color']='ICU'

q = df2_concat['label'].str.contains('CCU').fillna(False)
df2_concat.loc[q,'color']='ICU'

q = df2_concat['label'].str.contains('Surgery').fillna(False)
df2_concat.loc[q,'color']='Surgery'

q = df2_concat['label'].isin(['CABG']).fillna(False)
df2_concat.loc[q,'color']='CABG'

q = df2_concat['label'].str.contains('Cath').fillna(False)
df2_concat.loc[q,'color']='Outcome'

q = df2_concat['label'].isin(['sepsis']).fillna(False)
df2_concat.loc[q,'color']='sepsis'

df2_concat

In [ ]:



fig = px.timeline(df2_concat.iloc[:], y='id', x_start='intime', x_end = 'outtime', color='color',
        hover_data=['hadm_id','detail'])
fig.update_traces(width=0.2)
fig.show()
fig.write_html(ADD_DATA+'sepsis timeline.html')



In [ ]:
df=pd.read_csv(ADD_DATA+'d_labitems.csv').query("label.str.contains('lactate',case=False)").drop_duplicates('itemid')
df

# visualize a single patient

In [ ]:


new_id = 20508757
dict_le_bio = {
#51003:	'Troponin T',
#50910:	'Creatine Kinase (CK)',
#50911:	'Creatine Kinase, MB Isoenzyme',
50813:	'Lactate',
}

dict_che_bio = {
#227429:	'Troponin-T',
220045:	'Heart Rate',	
}
#dict_che_bio = {}
q_le = (df_le['hadm_id']==new_id) & ( df_le['itemid'].isin(list(dict_le_bio.keys())) )
df2_le = df_le[q_le]
df2_le['label'] = df2_le['itemid'].replace(dict_le_bio)

q_che = (df_che['hadm_id']==new_id) & ( df_che['itemid'].isin(list(dict_che_bio.keys())) )
df2_che = df_che[q_che]
df2_che['label'] = df2_che['itemid'].replace(dict_che_bio)

df2_che

df_bio = pd.concat([df2_le[['hadm_id','itemid','label','charttime', 'valuenum']],
    df2_che[['hadm_id','itemid','label','charttime', 'valuenum']],]
    )
df_bio = pd.merge(df_bio,df_admissions[['hadm_id','admittime']], on='hadm_id')
df_bio ['time'] = (df_bio ['charttime'] - df_bio ['admittime']) + date_origin

df_bio = df_bio.sort_values(by=['itemid','time'])
#px.line(df_bio,x='time',y='valuenum',color='label')
# df=pd.read_csv(ADD_DATA+'d_labitems.csv').query("label.str.contains('creatin',case=False)").drop_duplicates('itemid')
# df

df_bio['time'] = (df_bio['time'] - date_origin) / np.timedelta64(1,'h')

list_bio = df_bio['itemid'].unique().tolist()



fig = make_subplots(rows=len(list_bio)+1, cols=1,
            shared_xaxes=True,
            subplot_titles=[*df_bio['label'].unique(),'Time line']
            )

for i,itemid in enumerate(list_bio):
    df_temp = df_bio[df_bio['itemid']==itemid]
    fig.add_trace(
        go.Scatter(x=df_temp['time'], y=df_temp['valuenum'],
                    mode='lines+markers',
                    name=df_temp['label'].iloc[0],
        ),
        row=i+1, col=1
    )






In [ ]:


df_temp = df2_concat[df2_concat['hadm_id']==new_id]
df_temp['intime'] = (df_temp['intime'] - date_origin) / np.timedelta64(1,'h')
df_temp['outtime'] = (df_temp['outtime'] - date_origin) / np.timedelta64(1,'h')
#df_temp['id']= 'Events'

q_cu = df_temp['id']%1==0
q_events = ~q_cu




df_temp.loc[q_cu,'id'] = 'Care units'
df_temp.loc[q_events,'id'] = 'Events'

fig_temp = hk_psql.plot_timeline(df_temp, x_start="intime", x_end="outtime", y='id', color='color',hover_data=['hadm_id','detail'])
fig_temp.update_traces(width=0.2)
  
for trace in fig_temp.data:
    #print(trace)
    fig.add_trace(trace, row=len(list_bio)+1, col=1)



In [ ]:
fig.update_layout(title_text = f"id: {new_id}")
fig.show()


fig.write_html(ADD_DATA+f'fig_timeline_id{new_id}.html')


# Alignment by outcome
aligned with respect to outcome

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

df2_concat.dtypes
outcome = 'Cath Lab (Sent)'
df_concat_sub = df2_concat.query("label=='Cath Lab (Sent)'")

hadm_ids_selected = df_concat_sub['hadm_id'].unique()
df_bio_sub = df_bio[df_bio['hadm_id'].isin(hadm_ids_selected)]




q_le = (df_le['hadm_id'].isin(hadm_ids_selected)) & ( df_le['itemid'].isin(list(dict_le_bio.keys())) )
df2_le = df_le[q_le]
df2_le['label'] = df2_le['itemid'].replace(dict_le_bio)


df2_le = pd.merge(df2_le,df_admissions[['hadm_id','admittime']], on='hadm_id')
df2_le ['time'] = (df2_le ['charttime'] - df2_le ['admittime']) + date_origin

df2_le = df2_le.sort_values(by=['itemid','time'])
df2_le['time'] = (df2_le['time'] - date_origin) / np.timedelta64(1,'h')

df = pd.merge(df2_le, df_concat_sub[['hadm_id','intime']],on='hadm_id')
print(df.dtypes)
print(type(date_origin))
df['intime'] = ( df['intime'] -date_origin) / np.timedelta64(1,'h')
df['aligned'] = df['time']-df['intime'] 
df


In [ ]:
list_bio = df['itemid'].unique().tolist()

print(list_bio)
print([*df['label'].unique(),'Time line'])

fig = make_subplots(rows=len(list_bio)+1, cols=1,
            shared_xaxes=True,
            subplot_titles=[*df['label'].unique(),'Time line']
            )

for i,itemid in enumerate(list_bio):
    df_temp = df[df['itemid']==itemid]
    new_trace = go.Scatter(x=df_temp['aligned'], y=df_temp['valuenum'],
                    mode='markers',
                    name=df_temp['label'].iloc[0]
        )
    fig.add_trace( new_trace,
        row=i+1, col=1
    )

fig.show()


fig.write_html(ADD_DATA+f'fig_batch.html')

In [ ]:
for i,itemid in enumerate(list_bio):
    df_temp = df[df['itemid']==itemid]
    print(len(df_temp['hadm_id'].unique()))
    df_temp['hadm_id'] = df_temp['hadm_id'].astype(str)
    fig = px.scatter(df_temp,x='aligned', y='valuenum',#color='hadm_id',
                marginal_x="histogram",
                marginal_y="histogram",
                title=df_temp['label'].iloc[0]
                
    )
    fig.show()


In [ ]:
for i,itemid in enumerate(list_bio):
    df_temp = df[df['itemid']==itemid]
    print(len(df_temp['hadm_id'].unique()))
    df_temp['hadm_id'] = df_temp['hadm_id'].astype(str)
    fig = px.scatter(df_temp,x='time', y='valuenum',#color='hadm_id',
                marginal_x="histogram",
                marginal_y="histogram",
                title=df_temp['label'].iloc[0]
                
    )
    fig.show()

In [ ]:
df_che.dtypes
print(df_che[df_che['itemid'].isin(list_itemids)])
df_che

In [ ]:
df_che_main = df_che.copy()

In [ ]:
df_che[df_che['itemid']==228445]


In [ ]:
# read desired itemids from csv files

ADD_DATA = "C:\\DATA\\data\\raw\\mimic4\\lookup\\"
df= pd.read_csv(ADD_DATA+'d_lookup_le.csv')
le_itemdids = df['itemid']


df= pd.read_csv(ADD_DATA+'d_lookup_che.csv')
che_itemdids = df['itemid']


grid='1H'


In [ ]:
df_le = df_le_main
df_che = df_che_main
print(len(df_demo['hadm_id'].unique()))
print(len(df_le['hadm_id'].unique()))
print(len(df_icustay['stay_id'].unique()))

icustay_ids = len(df_icustay['stay_id'].unique())

df_le = df_le[df_le['itemid'].isin(le_itemdids)]
df_che = df_che[df_che['itemid'].isin(che_itemdids)]

print(df_le['itemid'].value_counts())


df_le = pd.merge(df_le[['hadm_id','itemid','charttime','valuenum']],
         df_icustay, on='hadm_id',how='inner')

df_che = pd.merge(df_che[['hadm_id','itemid','charttime','valuenum']],
         df_icustay, on='hadm_id',how='inner')

In [ ]:
# filtering icu stays, reset charttime to start from zero (icu admission)

df_le['temp1'] = round((df_le['charttime'] - df_le['intime']) / np.timedelta64(1, 's')/3600,2)
df_le['temp2'] = round((df_le['outtime'] - df_le['charttime']) / np.timedelta64(1, 's')/3600,2)
df_le = df_le[ (df_le['temp1']>0) & (df_le['temp2']>0) ]
df_le['charttime'] = df_le['temp1']
df_le = df_le.sort_values(['stay_id','itemid', 'charttime'])

df_che['temp1'] = round((df_che['charttime'] - df_che['intime']) / np.timedelta64(1, 's')/3600,2)
df_che['temp2'] = round((df_che['outtime'] - df_che['charttime']) / np.timedelta64(1, 's')/3600,2)
df_che = df_che[ (df_che['temp1']>0) & (df_che['temp2']>0) ]
df_che['charttime'] = df_che['temp1']
df_che = df_che.sort_values(['stay_id','itemid', 'charttime'])




In [ ]:
#convert charttime(int) to hours, and resample and forward fill imputation

from datetime import datetime


df_le = df_le[['stay_id','itemid','charttime','valuenum']][:]
df1=df_le
df_le['charttime'] = datetime(2000,1,1,0,0,0) + df_le.charttime.astype('timedelta64[h]')
df_le = df_le.set_index('charttime')
# print(df_le.index)
df_le= df_le.groupby(['stay_id','itemid']).resample(grid)['valuenum'].last().reset_index(drop=False)
df_le['valuenum']= df_le.groupby(['stay_id','itemid'])['valuenum'].fillna(method='ffill')
df_le = df_le.sort_values(['stay_id','itemid', 'charttime'])



df_che = df_che[['stay_id','itemid','charttime','valuenum']][:]
df1=df_che
df_che['charttime'] = datetime(2000,1,1,0,0,0) + df_che.charttime.astype('timedelta64[h]')
df_che = df_che.set_index('charttime')
# print(df_che.index)
df_che= df_che.groupby(['stay_id','itemid']).resample(grid)['valuenum'].last().reset_index(drop=False)
df_che['valuenum']= df_che.groupby(['stay_id','itemid'])['valuenum'].fillna(method='ffill')
df_che = df_che.sort_values(['stay_id','itemid', 'charttime'])


In [ ]:
# pivot dataframes so that each measurement is a column
# concatenate df_le and df_che

df_lee = df_le.set_index(['charttime','stay_id'])
df_lee = df_lee.pivot(columns = 'itemid', values='valuenum')

df_chee = df_che.set_index(['charttime','stay_id'])
df_chee = df_chee.pivot(columns = 'itemid', values='valuenum')

df = pd.concat([df_lee, df_chee], axis=1)
# df = df_chee


df = df.reset_index(drop=False)
df = df.sort_values(['stay_id', 'charttime'])


print(df.isnull().sum())
print(len(df))


print(df.dropna().isnull().sum())
print(len(df.dropna()))

df = df.dropna(subset=[51003])


In [ ]:
df
print(df['stay_id'].value_counts())
sample_id = df['stay_id'].value_counts().index[0]
print(sample_id)

In [ ]:
# plot before normalization

stay_ids = df_icustay['stay_id'].unique().tolist()
# print(df['stay_id'].value_counts())


df_temp = df[df['stay_id']==sample_id ]
fig = px.line(df_temp, x='charttime', y=[220045, 220210, 51003], markers=True)
fig.show()


# df_temp = df_le[df_le['stay_id']==39132344    ]
# fig = px.line(df_temp, x='charttime', y='valuenum', color='itemid', markers=True)
# fig.show()


# df_temp = df_che[df_che['stay_id']==39132344    ]
# fig = px.line(df_temp, x='charttime', y='valuenum', color='itemid', markers=True)
# fig.show()

In [ ]:
df_temp

In [ ]:
# normalizing the data

from sklearn.preprocessing import MinMaxScaler
# define min max scaler
scaler = MinMaxScaler()
# transform data
df_temp = df_temp.set_index(['charttime','stay_id'])

df_temp = pd.DataFrame(scaler.fit_transform(df_temp), index=df_temp.index, columns=df_temp.columns)
df_temp = df_temp.reset_index(drop=False)


In [ ]:
# plot after normalization

print(df_temp.columns)
fig = px.line(df_temp, x='charttime', y=[       220045,    220210,  ], markers=True)
fig.show()


In [ ]:
from sklearn.model_selection import GroupShuffleSplit
splitter = GroupShuffleSplit(test_size=.20, n_splits=1, random_state = 7)
split = splitter.split(df, groups=df['stay_id'])
train_inds, test_inds = next(split)

df_train = df.iloc[train_inds]
df_test = df.iloc[test_inds]

print(len(df['stay_id'].unique()))
print(len(df_train['stay_id'].unique()))
print(len(df_test['stay_id'].unique()))

In [ ]:
def split_series(series, n_past, n_future):
  #
  # n_past ==> no of past observations
  #
  # n_future ==> no of future observations 
  #
  X, y = list(), list()
  for window_start in range(len(series)):
    past_end = window_start + n_past
    future_end = past_end + n_future
    if future_end > len(series):
      break
    # slicing the past and future parts of the window
    past, future = series[window_start:past_end, :], series[past_end:future_end, :]
    X.append(past)
    y.append(future)
  return np.array(X), np.array(y)

n_past = 3  # * grid
n_future = 2 # * grid
n_features = 2




# convert train dataframe to numpy
stay_ids = df_train['stay_id'].unique()
list_X = []
list_y = []
for i in stay_ids:
  df_temp = df_train[df_train['stay_id']==i].drop(columns=['stay_id','charttime']).to_numpy()
  X_train, y_train = split_series(df_temp ,n_past, n_future)
  
  list_X.append(X_train)
  list_y.append(y_train)

list_X = [i for i in list_X if i.ndim==3]
list_y = [i for i in list_y if i.ndim==3]

X_train=np.concatenate(list_X, axis = 0)
y_train=np.concatenate(list_y, axis = 0)



# convert test dataframe to numpy
stay_ids = df_test['stay_id'].unique()
list_X = []
list_y = []
for i in stay_ids:
  df_temp = df_test[df_test['stay_id']==i].drop(columns=['stay_id','charttime']).to_numpy()
  X_test, y_test = split_series(df_temp ,n_past, n_future)

  list_X.append(X_test)
  list_y.append(y_test)

list_X = [i for i in list_X if i.ndim==3]
list_y = [i for i in list_y if i.ndim==3]

X_test=np.concatenate(list_X, axis = 0)
y_test=np.concatenate(list_y, axis = 0)


print(f""" 
X_train.shape = {X_train.shape}
y_train.shape = {y_train.shape}
X_test.shape = {X_test.shape}
y_test.shape = {y_test.shape}
 """)


In [ ]:

import tensorflow as tf
import os
n_features=X_train.shape[-1]
n_past=X_train.shape[-2]
n_future=y_train.shape[-1]

print(X_train.shape, X_test.shape)

In [ ]:
encoder = tf.keras.models.Sequential([
    # tf.keras.layers.LSTM(100, return_state=True, input_shape = (n_past,n_features)),
    tf.keras.layers.Flatten(input_shape=[3, 2]),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(30, activation="relu"),
])

In [ ]:
# E1D1
# n_features ==> no of features at each timestep in the data.
#
encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_l1 = tf.keras.layers.LSTM(100, return_state=True)
encoder_outputs1 = encoder_l1(encoder_inputs)

encoder_states1 = encoder_outputs1[1:]

#
decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs1[0])

#
decoder_l1 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_outputs1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_l1)

#
model_e1d1 = tf.keras.models.Model(encoder_inputs,decoder_outputs1)

#
model_e1d1.summary()

In [ ]:
reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)
model_e1d1.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
history_e1d1=model_e1d1.fit(X_train,y_train,epochs=25,validation_data=(X_test,y_test),batch_size=32,verbose=1,callbacks=[reduce_lr])


In [ ]:
print(history_e1d1.history['loss'])
plt.plot(history_e1d1.history['loss'])
plt.plot(history_e1d1.history['val_loss'])
plt.title("E1D1 Model Loss")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Train', 'Valid'])
plt.show()

In [ ]:
pred1_e1d1=model_e1d1.predict(X_test)
# pred1_e2d2=model_e2d2.predict(X_test)

pred_e1d1=model_e1d1.predict(X_train)
# pred_e2d2=model_e2d2.predict(X_train)

In [ ]:
print(pred1_e1d1.shape)
print(y_test.shape)
temp_test = y_test.reshape(-1,2)
temp_pred = pred1_e1d1.reshape(-1,2)


print(temp_pred.shape)
df_temp = pd.DataFrame(temp_pred[:1000,:],columns=[       220045,    220210,  ])
fig = px.line(df_temp, y=[       220045,    220210,  ], markers=True)

df_temp = pd.DataFrame(temp_test[:1000,:],columns=[       220045,    220210,  ])
fig = px.line(df_temp, y=[       220045,    220210,  ], markers=True)


fig.show()

In [ ]:
normal_df = df
RANDOM_SEED = 1374

In [ ]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(
  normal_df,
  test_size=0.15,
  random_state=RANDOM_SEED
)
val_df, test_df = train_test_split(
  val_df,
  test_size=0.33,
  random_state=RANDOM_SEED
)

In [ ]:
# Data preprocessing
def create_dataset(df):
  sequences = df.astype(np.float32).to_numpy().tolist()
  dataset = [torch.tensor(s).unsqueeze(1).float() for s in sequences]
  n_seq, seq_len, n_features = torch.stack(dataset).shape
  return dataset, seq_len, n_features


train_dataset, seq_len, n_features = create_dataset(train_df)
val_dataset, _, _ = create_dataset(val_df)
test_normal_dataset, _, _ = create_dataset(test_df)
test_anomaly_dataset, _, _ = create_dataset(anomaly_df)

In [ ]:
import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class Encoder(nn.Module):
  def __init__(self, seq_len, n_features, embedding_dim=64):
    super(Encoder, self).__init__()
    self.seq_len, self.n_features = seq_len, n_features
    self.embedding_dim, self.hidden_dim = embedding_dim, 2 * embedding_dim
    self.rnn1 = nn.LSTM(
      input_size=n_features,
      hidden_size=self.hidden_dim,
      num_layers=1,
      batch_first=True
    )
    self.rnn2 = nn.LSTM(
      input_size=self.hidden_dim,
      hidden_size=embedding_dim,
      num_layers=1,
      batch_first=True
    )
  def forward(self, x):
    x = x.reshape((1, self.seq_len, self.n_features))
    x, (_, _) = self.rnn1(x)
    x, (hidden_n, _) = self.rnn2(x)
    return hidden_n.reshape((self.n_features, self.embedding_dim))


class Decoder(nn.Module):
  def __init__(self, seq_len, input_dim=64, n_features=1):
    super(Decoder, self).__init__()
    self.seq_len, self.input_dim = seq_len, input_dim
    self.hidden_dim, self.n_features = 2 * input_dim, n_features
    self.rnn1 = nn.LSTM(
      input_size=input_dim,
      hidden_size=input_dim,
      num_layers=1,
      batch_first=True
    )
    self.rnn2 = nn.LSTM(
      input_size=input_dim,
      hidden_size=self.hidden_dim,
      num_layers=1,
      batch_first=True
    )
    self.output_layer = nn.Linear(self.hidden_dim, n_features)
  def forward(self, x):
    x = x.repeat(self.seq_len, self.n_features)
    x = x.reshape((self.n_features, self.seq_len, self.input_dim))
    x, (hidden_n, cell_n) = self.rnn1(x)
    x, (hidden_n, cell_n) = self.rnn2(x)
    x = x.reshape((self.seq_len, self.hidden_dim))
    return self.output_layer(x)

class RecurrentAutoencoder(nn.Module):
  def __init__(self, seq_len, n_features, embedding_dim=64):
    super(RecurrentAutoencoder, self).__init__()
    self.encoder = Encoder(seq_len, n_features, embedding_dim).to(device)
    self.decoder = Decoder(seq_len, embedding_dim, n_features).to(device)
  def forward(self, x):
    x = self.encoder(x)
    x = self.decoder(x)
    return x



In [ ]:
import copy

def train_model(model, train_dataset, val_dataset, n_epochs):
  optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
  criterion = nn.L1Loss(reduction='sum').to(device)
  history = dict(train=[], val=[])
  best_model_wts = copy.deepcopy(model.state_dict())
  best_loss = 10000.0
  for epoch in range(1, n_epochs + 1):
    model = model.train()
    train_losses = []
    for seq_true in train_dataset:
      optimizer.zero_grad()
      seq_true = seq_true.to(device)
      seq_pred = model(seq_true)
      loss = criterion(seq_pred, seq_true)
      loss.backward()
      optimizer.step()
      train_losses.append(loss.item())
    val_losses = []
    model = model.eval()
    with torch.no_grad():
      for seq_true in val_dataset:
        seq_true = seq_true.to(device)
        seq_pred = model(seq_true)
        loss = criterion(seq_pred, seq_true)
        val_losses.append(loss.item())
    train_loss = np.mean(train_losses)
    val_loss = np.mean(val_losses)
    history['train'].append(train_loss)
    history['val'].append(val_loss)
    if val_loss < best_loss:
      best_loss = val_loss
      best_model_wts = copy.deepcopy(model.state_dict())
    print(f'Epoch {epoch}: train loss {train_loss} val loss {val_loss}')
  model.load_state_dict(best_model_wts)
  return model.eval(), history

In [ ]:
model = RecurrentAutoencoder(seq_len, n_features, 128)
model = model.to(device)

model, history = train_model(
  model,
  train_dataset,
  val_dataset,
  n_epochs=150
)